In [1]:
from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Cropping2D, Dropout
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, History
from keras import backend as K
from keras.optimizers import Nadam
from keras.utils import multi_gpu_model

import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
list_gpus = K.tensorflow_backend._get_available_gpus()
print(list_gpus)


['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


In [3]:
import threading

In [4]:
import numpy as np
#from sklearn.model_selection import StratifiedShuffleSplit,ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [5]:
import sys
sys.path.insert(0, '/nfs/src')
from features import geopatches

In [6]:
import datetime
import os

In [7]:
import pandas as pd
import h5py

In [8]:
# Setup data seed
np.random.seed(3254)

In [9]:
# https://github.com/ternaus/kaggle_dstl_submission/

smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + K.binary_crossentropy(y_pred, y_true)
#    return -K.log(jaccard_coef(y_true, y_pred))

def save_model(model, cross, results_dir):
    json_string = model.to_json()
    if not os.path.isdir(os.path.join(results_dir,'cache')):
        os.makedirs(os.path.join(results_dir,'cache'))
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    open(os.path.join(results_dir,'cache', json_name), 'w').write(json_string)
    model.save_weights(os.path.join(results_dir,'cache', weight_name), overwrite=True)

def save_history(history, cross, results_dir):
    if not os.path.isdir(os.path.join(results_dir,'history')):
        os.makedirs(os.path.join(results_dir,'history'))
    filename = os.path.join(results_dir,'history') + '/history_' + cross + '.csv'
    pd.DataFrame(history.history).to_csv(filename, index=False)

def read_model(cross,results_dir):
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    model = model_from_json(open(os.path.join(results_dir,'cache', json_name)).read())
    model.load_weights(os.path.join(results_dir,'cache', weight_name))
    return model

In [10]:
def model_unet(img_cols, img_rows, nb_channels, img_crop):
    
    input_shape = (img_cols, img_rows, nb_channels)
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (inputs)
    c1 = BatchNormalization(axis=1)(c1)
    c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
    c1 = BatchNormalization(axis=1)(c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
    c2 = BatchNormalization(axis=1)(c2)
    c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
    c2 = BatchNormalization(axis=1)(c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
    c3 = BatchNormalization(axis=1)(c3)
    c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
    c3 = BatchNormalization(axis=1)(c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
    c4 = BatchNormalization(axis=1)(c4)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
    c4 = BatchNormalization(axis=1)(c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
    c5 = BatchNormalization(axis=1)(c5)
    c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (c5)
    c5 = BatchNormalization(axis=1)(c5)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
    c6 = BatchNormalization(axis=1)(c6)
    c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)
    c6 = BatchNormalization(axis=1)(c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
    c7 = BatchNormalization(axis=1)(c7)
    c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)
    c7 = BatchNormalization(axis=1)(c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
    c8 = BatchNormalization(axis=1)(c8)
    c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)
    c8 = BatchNormalization(axis=1)(c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = BatchNormalization(axis=1)(c9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)
    c9 = BatchNormalization(axis=1)(c9)
    
    #outputs = Conv2D(25, (1, 1), activation='sigmoid') (c9)
    c10 = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    outputs = Cropping2D(cropping=((img_crop, img_crop), (img_crop, img_crop))) (c10)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [11]:
#https://keunwoochoi.wordpress.com/2017/08/24/tip-fit_generator-in-keras-how-to-parallelise-correctly/

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def next(self):
        with self.lock:
            #return self.it.next()
            return next(self.it)


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def generate_minibatch(X_train, Y_train, batch_size):
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
        Y_batch = np.empty((batch_size, Y_train.shape[1], Y_train.shape[2], Y_train.shape[3]))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X_train.shape[0])
            image = X_train[idx]
            label = Y_train[idx]

            if np.random.choice(2) == 0:
                # Flip vertically
                image = np.flipud(image)
                label = np.flipud(label)
            if np.random.choice(2) == 0:
                # Flip horizontally
                image = np.fliplr(image)
                label = np.fliplr(label)

            num_rot = np.random.choice(4)
            image = np.rot90(image, num_rot)
            label = np.rot90(label, num_rot)

            X_batch[i] = image
            Y_batch[i] = label

        yield (X_batch, Y_batch)

In [12]:
def encode_binary_data(dict_replace):
    # Encode classes
    print(dict_replace)
    u,inv = np.unique(Y_raw,return_inverse = True)
    Y = np.array([dict_replace.get(x, 0.0) for x in u])[inv].reshape(Y_raw.shape)
    Y = Y[..., np.newaxis]
    X = X_raw
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=3254)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=3254)
    
    for data,name in zip([Y_train,Y_val,Y_test],['Y_train','Y_val','Y_test']):
        num_img = 0.0
        num_pixels = 0.0
        for x in range(data.shape[0]):
            unique, counts = np.unique(data[x], return_counts=True)
            if len(unique) > 1:
                #print(x, dict(zip(unique, counts)))
                num_img += 1
                num_pixels += dict(zip(unique, counts))[1]
        print ('composition with label != 0')
        print (name, num_img/data.shape[0], num_pixels/(data.shape[0] * data.shape[1] * data.shape[2]))
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [13]:
if __name__ == '__main__':
    
    results_dir = '/nfs/results/unet/relutest/80_80_16/results_prelim_unet_pl_veg'
    
    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140EVT_020818_UTM11_30_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')

    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        del ts_gen
        del X_train, X_val, X_test, Y_train, Y_val, Y_test
        del Y_predict
        K.clear_session()        
        print('\n')
        

unique labels found:
[3014 3092 3096 3097 3099 3110 3118 3129 3130 3152 3155 3181 3182 3183
 3184 3221 3292 3294 3296 3297 3298 3299 3900 3901 3902 3903 3904 3910
 3911 3912 3913 3914 3923 3924 3926 3927 3928 3929 3964 3984]
labels that do not meet threshold
[3014, 3096, 3155, 3181, 3182, 3183, 3221, 3901, 3911, 3923, 3924, 3926, 3927, 3928, 3964, 3984]
final labels
[3092, 3097, 3099, 3110, 3118, 3129, 3130, 3152, 3184, 3292, 3294, 3296, 3297, 3298, 3299, 3900, 3902, 3903, 3904, 3910, 3912, 3913, 3914, 3929]


label: 3092
{3092: 1.0}
composition with label != 0
Y_train 0.5410628019323671 0.1474718196457327
composition with label != 0
Y_val 0.6 0.19891964285714286
composition with label != 0
Y_test 0.5202312138728323 0.12354407514450867
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., callbacks=[<keras.ca..., validation_data=(array([[[..., steps_per_epoch=100, verbose=1, epochs=50)`


Epoch 1/50
100/100 [==============================] - 21s 205ms/step - loss: 4.8492 - binary_crossentropy: 0.4754 - jaccard_coef_int: 0.2173 - binary_accuracy: 0.8317 - val_loss: 5.8885 - val_binary_crossentropy: 1.4413 - val_jaccard_coef_int: 0.1251 - val_binary_accuracy: 0.7686
Epoch 2/50
100/100 [==============================] - 12s 121ms/step - loss: 4.0322 - binary_crossentropy: 0.5565 - jaccard_coef_int: 0.2752 - binary_accuracy: 0.8431 - val_loss: 3.9907 - val_binary_crossentropy: 0.7447 - val_jaccard_coef_int: 0.4153 - val_binary_accuracy: 0.8140
Epoch 3/50
100/100 [==============================] - 12s 119ms/step - loss: 3.9069 - binary_crossentropy: 0.6027 - jaccard_coef_int: 0.2799 - binary_accuracy: 0.8450 - val_loss: 4.2389 - val_binary_crossentropy: 0.6039 - val_jaccard_coef_int: 0.3039 - val_binary_accuracy: 0.8188
Epoch 4/50
100/100 [==============================] - 12s 118ms/step - loss: 3.4343 - binary_crossentropy: 0.6256 - jaccard_coef_int: 0.3295 - binary_accurac

100/100 [==============================] - 12s 119ms/step - loss: 2.2834 - binary_crossentropy: 0.8948 - jaccard_coef_int: 0.4853 - binary_accuracy: 0.9031 - val_loss: 3.0726 - val_binary_crossentropy: 1.3285 - val_jaccard_coef_int: 0.4771 - val_binary_accuracy: 0.8545
Epoch 31/50
100/100 [==============================] - 12s 120ms/step - loss: 2.2120 - binary_crossentropy: 0.8684 - jaccard_coef_int: 0.4955 - binary_accuracy: 0.9061 - val_loss: 3.3448 - val_binary_crossentropy: 1.4446 - val_jaccard_coef_int: 0.4557 - val_binary_accuracy: 0.8406
Epoch 32/50
100/100 [==============================] - 12s 119ms/step - loss: 2.2020 - binary_crossentropy: 0.8788 - jaccard_coef_int: 0.4884 - binary_accuracy: 0.9077 - val_loss: 6.4169 - val_binary_crossentropy: 1.9276 - val_jaccard_coef_int: 0.0368 - val_binary_accuracy: 0.8045
Epoch 33/50
100/100 [==============================] - 12s 120ms/step - loss: 2.2587 - binary_crossentropy: 0.9108 - jaccard_coef_int: 0.4859 - binary_accuracy: 0.904

Epoch 7/50
100/100 [==============================] - 10s 95ms/step - loss: 2.7722 - binary_crossentropy: 0.0624 - jaccard_coef_int: 0.0931 - binary_accuracy: 0.9885 - val_loss: 5.1979 - val_binary_crossentropy: 0.0857 - val_jaccard_coef_int: 0.0062 - val_binary_accuracy: 0.9885
Epoch 8/50
100/100 [==============================] - 9s 95ms/step - loss: 2.5657 - binary_crossentropy: 0.0653 - jaccard_coef_int: 0.1037 - binary_accuracy: 0.9896 - val_loss: 8.3328 - val_binary_crossentropy: 0.0926 - val_jaccard_coef_int: 1.6637e-14 - val_binary_accuracy: 0.9882
Epoch 9/50
100/100 [==============================] - 10s 96ms/step - loss: 2.6924 - binary_crossentropy: 0.0764 - jaccard_coef_int: 0.0940 - binary_accuracy: 0.9881 - val_loss: 3.6346 - val_binary_crossentropy: 0.0926 - val_jaccard_coef_int: 0.0330 - val_binary_accuracy: 0.9879
Epoch 10/50
100/100 [==============================] - 9s 93ms/step - loss: 4.2961 - binary_crossentropy: 0.0595 - jaccard_coef_int: 0.0380 - binary_accuracy

...

Epoch 10/50
100/100 [==============================] - 11s 107ms/step - loss: 3.4323 - binary_crossentropy: 0.5290 - jaccard_coef_int: 0.1170 - binary_accuracy: 0.9219 - val_loss: 4.3113 - val_binary_crossentropy: 0.6112 - val_jaccard_coef_int: 0.0542 - val_binary_accuracy: 0.9129
Epoch 11/50
100/100 [==============================] - 11s 111ms/step - loss: 3.3493 - binary_crossentropy: 0.5568 - jaccard_coef_int: 0.1262 - binary_accuracy: 0.9219 - val_loss: 4.2070 - val_binary_crossentropy: 0.6181 - val_jaccard_coef_int: 0.0613 - val_binary_accuracy: 0.9122
Epoch 12/50
100/100 [==============================] - 10s 101ms/step - loss: 3.2407 - binary_crossentropy: 0.5584 - jaccard_coef_int: 0.1356 - binary_accuracy: 0.9241 - val_loss: 5.2306 - val_binary_crossentropy: 1.2928 - val_jaccard_coef_int: 0.0882 - val_binary_accuracy: 0.8261
Epoch 13/50
100/100 [==============================] - 10s 102ms/step - loss: 3.1597 - binary_crossentropy: 0.5558 - jaccard_coef_int: 0.1414 - binary_acc

100/100 [==============================] - 10s 99ms/step - loss: 2.1735 - binary_crossentropy: 0.5776 - jaccard_coef_int: 0.2805 - binary_accuracy: 0.9443 - val_loss: 3.9636 - val_binary_crossentropy: 0.8246 - val_jaccard_coef_int: 0.0683 - val_binary_accuracy: 0.9200
Epoch 40/50
100/100 [==============================] - 10s 102ms/step - loss: 2.1688 - binary_crossentropy: 0.5759 - jaccard_coef_int: 0.2795 - binary_accuracy: 0.9448 - val_loss: 3.9216 - val_binary_crossentropy: 0.7192 - val_jaccard_coef_int: 0.0584 - val_binary_accuracy: 0.9321
Epoch 41/50
100/100 [==============================] - 10s 97ms/step - loss: 2.1455 - binary_crossentropy: 0.5834 - jaccard_coef_int: 0.2869 - binary_accuracy: 0.9444 - val_loss: 3.9734 - val_binary_crossentropy: 0.7158 - val_jaccard_coef_int: 0.0552 - val_binary_accuracy: 0.9327
Epoch 42/50
100/100 [==============================] - 10s 104ms/step - loss: 2.1346 - binary_crossentropy: 0.5824 - jaccard_coef_int: 0.2875 - binary_accuracy: 0.9451 

Epoch 16/50
100/100 [==============================] - 10s 99ms/step - loss: 3.1348 - binary_crossentropy: 0.5981 - jaccard_coef_int: 0.1366 - binary_accuracy: 0.9296 - val_loss: 4.5049 - val_binary_crossentropy: 0.6043 - val_jaccard_coef_int: 0.0342 - val_binary_accuracy: 0.9295
Epoch 17/50
100/100 [==============================] - 10s 99ms/step - loss: 3.0145 - binary_crossentropy: 0.5768 - jaccard_coef_int: 0.1449 - binary_accuracy: 0.9333 - val_loss: 4.4085 - val_binary_crossentropy: 0.8478 - val_jaccard_coef_int: 0.0604 - val_binary_accuracy: 0.9001
Epoch 18/50
100/100 [==============================] - 10s 103ms/step - loss: 2.9991 - binary_crossentropy: 0.5946 - jaccard_coef_int: 0.1481 - binary_accuracy: 0.9329 - val_loss: 4.3137 - val_binary_crossentropy: 0.6001 - val_jaccard_coef_int: 0.0401 - val_binary_accuracy: 0.9313
Epoch 19/50
100/100 [==============================] - 11s 108ms/step - loss: 2.9304 - binary_crossentropy: 0.5964 - jaccard_coef_int: 0.1559 - binary_accur

100/100 [==============================] - 10s 103ms/step - loss: 2.1144 - binary_crossentropy: 0.5975 - jaccard_coef_int: 0.2751 - binary_accuracy: 0.9490 - val_loss: 4.0783 - val_binary_crossentropy: 0.8867 - val_jaccard_coef_int: 0.0559 - val_binary_accuracy: 0.9251
Epoch 46/50
100/100 [==============================] - 10s 105ms/step - loss: 2.1162 - binary_crossentropy: 0.6023 - jaccard_coef_int: 0.2751 - binary_accuracy: 0.9490 - val_loss: 4.2947 - val_binary_crossentropy: 0.8799 - val_jaccard_coef_int: 0.0445 - val_binary_accuracy: 0.9255
Epoch 47/50
100/100 [==============================] - 10s 96ms/step - loss: 2.0714 - binary_crossentropy: 0.5925 - jaccard_coef_int: 0.2840 - binary_accuracy: 0.9498 - val_loss: 4.1855 - val_binary_crossentropy: 0.7864 - val_jaccard_coef_int: 0.0471 - val_binary_accuracy: 0.9294
Epoch 48/50
100/100 [==============================] - 10s 95ms/step - loss: 2.0527 - binary_crossentropy: 0.5948 - jaccard_coef_int: 0.2885 - binary_accuracy: 0.9498 

Epoch 22/50
100/100 [==============================] - 9s 95ms/step - loss: 2.4849 - binary_crossentropy: 0.0904 - jaccard_coef_int: 0.1603 - binary_accuracy: 0.9873 - val_loss: 8.6550 - val_binary_crossentropy: 0.0054 - val_jaccard_coef_int: 0.6250 - val_binary_accuracy: 0.9993
Epoch 23/50
100/100 [==============================] - 10s 96ms/step - loss: 2.6765 - binary_crossentropy: 0.0958 - jaccard_coef_int: 0.1671 - binary_accuracy: 0.9876 - val_loss: 8.1217 - val_binary_crossentropy: 1.1243 - val_jaccard_coef_int: 0.0038 - val_binary_accuracy: 0.8480
Epoch 24/50
100/100 [==============================] - 10s 98ms/step - loss: 2.7869 - binary_crossentropy: 0.0887 - jaccard_coef_int: 0.1396 - binary_accuracy: 0.9871 - val_loss: 6.1856 - val_binary_crossentropy: 0.0321 - val_jaccard_coef_int: 0.2896 - val_binary_accuracy: 0.9951
Epoch 25/50
100/100 [==============================] - 10s 96ms/step - loss: 2.4394 - binary_crossentropy: 0.0868 - jaccard_coef_int: 0.1710 - binary_accuracy

In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/relutest/80_80_16/results_prelim_unet_pl_fire'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140FBFM13_3030_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        del ts_gen
        del X_train, X_val, X_test, Y_train, Y_val, Y_test
        del Y_predict
        K.clear_session()          
        print('\n')

unique labels found:
[ 1  2  4  5  7  8  9 91 93 98 99]
labels that do not meet threshold
[7, 93]
final labels
[1, 2, 4, 5, 8, 9, 91, 98, 99]


label: 1
{1: 1.0}
composition with label != 0
Y_train 0.8582930756843801 0.2713395732689211
composition with label != 0
Y_val 0.8428571428571429 0.2126607142857143
composition with label != 0
Y_test 0.861271676300578 0.2920628612716763
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., callbacks=[<keras.ca..., validation_data=(array([[[..., steps_per_epoch=100, verbose=1, epochs=50)`


Epoch 1/50
100/100 [==============================] - 16s 161ms/step - loss: 6.0835 - binary_crossentropy: 0.7204 - jaccard_coef_int: 0.2159 - binary_accuracy: 0.7439 - val_loss: 5.0283 - val_binary_crossentropy: 0.9437 - val_jaccard_coef_int: 0.1727 - val_binary_accuracy: 0.8005
Epoch 2/50
100/100 [==============================] - 10s 96ms/step - loss: 4.7364 - binary_crossentropy: 0.8265 - jaccard_coef_int: 0.3586 - binary_accuracy: 0.7814 - val_loss: 7.3281 - val_binary_crossentropy: 1.1706 - val_jaccard_coef_int: 0.0169 - val_binary_accuracy: 0.7850
Epoch 3/50
100/100 [==============================] - 10s 96ms/step - loss: 4.4453 - binary_crossentropy: 0.9225 - jaccard_coef_int: 0.3865 - binary_accuracy: 0.7886 - val_loss: 7.4435 - val_binary_crossentropy: 1.2088 - val_jaccard_coef_int: 0.0162 - val_binary_accuracy: 0.7760
Epoch 4/50
100/100 [==============================] - 10s 96ms/step - loss: 4.3417 - binary_crossentropy: 1.0011 - jaccard_coef_int: 0.3902 - binary_accuracy: 

100/100 [==============================] - 9s 95ms/step - loss: 2.8757 - binary_crossentropy: 1.3680 - jaccard_coef_int: 0.5578 - binary_accuracy: 0.8570 - val_loss: 3.8971 - val_binary_crossentropy: 1.8243 - val_jaccard_coef_int: 0.3657 - val_binary_accuracy: 0.8195
Epoch 31/50
100/100 [==============================] - 9s 93ms/step - loss: 2.9328 - binary_crossentropy: 1.4138 - jaccard_coef_int: 0.5527 - binary_accuracy: 0.8540 - val_loss: 4.4265 - val_binary_crossentropy: 2.0570 - val_jaccard_coef_int: 0.3705 - val_binary_accuracy: 0.7859
Epoch 32/50
100/100 [==============================] - 10s 96ms/step - loss: 2.8314 - binary_crossentropy: 1.3801 - jaccard_coef_int: 0.5675 - binary_accuracy: 0.8587 - val_loss: 3.9259 - val_binary_crossentropy: 1.8681 - val_jaccard_coef_int: 0.3263 - val_binary_accuracy: 0.8245
Epoch 33/50
100/100 [==============================] - 10s 96ms/step - loss: 2.8282 - binary_crossentropy: 1.3934 - jaccard_coef_int: 0.5625 - binary_accuracy: 0.8595 - va

Epoch 7/50
100/100 [==============================] - 9s 93ms/step - loss: 4.7547 - binary_crossentropy: 0.7192 - jaccard_coef_int: 0.0731 - binary_accuracy: 0.8711 - val_loss: 4.8047 - val_binary_crossentropy: 1.2506 - val_jaccard_coef_int: 0.1317 - val_binary_accuracy: 0.8299
Epoch 8/50
100/100 [==============================] - 9s 95ms/step - loss: 4.7008 - binary_crossentropy: 0.7024 - jaccard_coef_int: 0.0734 - binary_accuracy: 0.8732 - val_loss: 13.9473 - val_binary_crossentropy: 10.7087 - val_jaccard_coef_int: 0.1138 - val_binary_accuracy: 0.2691
Epoch 9/50
100/100 [==============================] - 9s 93ms/step - loss: 4.6417 - binary_crossentropy: 0.6980 - jaccard_coef_int: 0.0817 - binary_accuracy: 0.8707 - val_loss: 11.3820 - val_binary_crossentropy: 6.7375 - val_jaccard_coef_int: 0.1137 - val_binary_accuracy: 0.4271
Epoch 10/50
100/100 [==============================] - 9s 93ms/step - loss: 4.3594 - binary_crossentropy: 0.7603 - jaccard_coef_int: 0.1157 - binary_accuracy: 0

...

100/100 [==============================] - 9s 94ms/step - loss: 1.5889 - binary_crossentropy: 0.7663 - jaccard_coef_int: 0.7688 - binary_accuracy: 0.9174 - val_loss: 3.0999 - val_binary_crossentropy: 1.5514 - val_jaccard_coef_int: 0.6191 - val_binary_accuracy: 0.8365
Epoch 38/50
100/100 [==============================] - 9s 94ms/step - loss: 1.5649 - binary_crossentropy: 0.7601 - jaccard_coef_int: 0.7738 - binary_accuracy: 0.9184 - val_loss: 3.0970 - val_binary_crossentropy: 1.5434 - val_jaccard_coef_int: 0.6087 - val_binary_accuracy: 0.8376
Epoch 39/50
100/100 [==============================] - 10s 100ms/step - loss: 1.5579 - binary_crossentropy: 0.7636 - jaccard_coef_int: 0.7729 - binary_accuracy: 0.9190 - val_loss: 3.2346 - val_binary_crossentropy: 1.6167 - val_jaccard_coef_int: 0.6050 - val_binary_accuracy: 0.8297
Epoch 40/50
100/100 [==============================] - 10s 95ms/step - loss: 1.5263 - binary_crossentropy: 0.7542 - jaccard_coef_int: 0.7781 - binary_accuracy: 0.9205 - v

Epoch 14/50
100/100 [==============================] - 9s 93ms/step - loss: 0.6470 - binary_crossentropy: 0.0452 - jaccard_coef_int: 0.6425 - binary_accuracy: 0.9931 - val_loss: 1.0865 - val_binary_crossentropy: 0.1000 - val_jaccard_coef_int: 0.4275 - val_binary_accuracy: 0.9860
Epoch 15/50
100/100 [==============================] - 9s 94ms/step - loss: 0.7100 - binary_crossentropy: 0.0530 - jaccard_coef_int: 0.6292 - binary_accuracy: 0.9920 - val_loss: 1.0866 - val_binary_crossentropy: 0.1008 - val_jaccard_coef_int: 0.4279 - val_binary_accuracy: 0.9860
Epoch 16/50
100/100 [==============================] - 9s 95ms/step - loss: 0.5977 - binary_crossentropy: 0.0447 - jaccard_coef_int: 0.6592 - binary_accuracy: 0.9934 - val_loss: 1.0825 - val_binary_crossentropy: 0.1045 - val_jaccard_coef_int: 0.4279 - val_binary_accuracy: 0.9860
Epoch 17/50
100/100 [==============================] - 9s 93ms/step - loss: 0.6209 - binary_crossentropy: 0.0514 - jaccard_coef_int: 0.6480 - binary_accuracy: 0

100/100 [==============================] - 10s 96ms/step - loss: 0.6048 - binary_crossentropy: 0.0750 - jaccard_coef_int: 0.6469 - binary_accuracy: 0.9927 - val_loss: 1.0815 - val_binary_crossentropy: 0.1623 - val_jaccard_coef_int: 0.4270 - val_binary_accuracy: 0.9859
Epoch 44/50
100/100 [==============================] - 9s 93ms/step - loss: 0.5936 - binary_crossentropy: 0.0692 - jaccard_coef_int: 0.6571 - binary_accuracy: 0.9932 - val_loss: 1.0727 - val_binary_crossentropy: 0.1521 - val_jaccard_coef_int: 0.4281 - val_binary_accuracy: 0.9862
Epoch 45/50
100/100 [==============================] - 9s 94ms/step - loss: 0.5505 - binary_crossentropy: 0.0604 - jaccard_coef_int: 0.6799 - binary_accuracy: 0.9940 - val_loss: 1.1103 - val_binary_crossentropy: 0.1524 - val_jaccard_coef_int: 0.4133 - val_binary_accuracy: 0.9860
Epoch 46/50
100/100 [==============================] - 9s 94ms/step - loss: 0.6171 - binary_crossentropy: 0.0725 - jaccard_coef_int: 0.6498 - binary_accuracy: 0.9931 - val

Epoch 20/50
100/100 [==============================] - 9s 94ms/step - loss: 3.7225 - binary_crossentropy: 0.0969 - jaccard_coef_int: 0.0476 - binary_accuracy: 0.9782 - val_loss: 4.7693 - val_binary_crossentropy: 0.1130 - val_jaccard_coef_int: 0.0068 - val_binary_accuracy: 0.9807
Epoch 21/50
100/100 [==============================] - 9s 94ms/step - loss: 3.5344 - binary_crossentropy: 0.0835 - jaccard_coef_int: 0.0529 - binary_accuracy: 0.9824 - val_loss: 3.9213 - val_binary_crossentropy: 0.1219 - val_jaccard_coef_int: 0.0192 - val_binary_accuracy: 0.9822
Epoch 22/50
100/100 [==============================] - 9s 94ms/step - loss: 3.5223 - binary_crossentropy: 0.1031 - jaccard_coef_int: 0.0544 - binary_accuracy: 0.9780 - val_loss: 3.0588 - val_binary_crossentropy: 0.1774 - val_jaccard_coef_int: 0.0742 - val_binary_accuracy: 0.9739
Epoch 23/50
100/100 [==============================] - 9s 94ms/step - loss: 3.2389 - binary_crossentropy: 0.0916 - jaccard_coef_int: 0.0657 - binary_accuracy: 0

100/100 [==============================] - 9s 95ms/step - loss: 3.5113 - binary_crossentropy: 0.0969 - jaccard_coef_int: 0.0601 - binary_accuracy: 0.9862 - val_loss: 4.7429 - val_binary_crossentropy: 0.1158 - val_jaccard_coef_int: 0.0108 - val_binary_accuracy: 0.9829
Epoch 50/50
100/100 [==============================] - 9s 95ms/step - loss: 3.5640 - binary_crossentropy: 0.1150 - jaccard_coef_int: 0.0542 - binary_accuracy: 0.9830 - val_loss: 3.8334 - val_binary_crossentropy: 0.1459 - val_jaccard_coef_int: 0.0293 - val_binary_accuracy: 0.9793
0.9892124277456648
             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99   1100808
        1.0       0.02      0.01      0.01      6392

avg / total       0.99      0.99      0.99   1107200





In [15]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/relutest/80_80_16/results_prelim_unet_pl_landcover'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/escondido_landcover_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        del ts_gen
        del X_train, X_val, X_test, Y_train, Y_val, Y_test
        del Y_predict
        K.clear_session()          
        print('\n')

unique labels found:
[11 21 22 23 24 31 42 43 52 71 81 82 90 95]
labels that do not meet threshold
[42, 81, 95]
final labels
[11, 21, 22, 23, 24, 31, 43, 52, 71, 82, 90]


label: 11
{11: 1.0}
composition with label != 0
Y_train 0.06924315619967794 0.017379227053140097
composition with label != 0
Y_val 0.05714285714285714 0.019285714285714285
composition with label != 0
Y_test 0.046242774566473986 0.011065751445086706
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., callbacks=[<keras.ca..., validation_data=(array([[[..., steps_per_epoch=100, verbose=1, epochs=50)`


Epoch 1/50
100/100 [==============================] - 16s 164ms/step - loss: 1.8412 - binary_crossentropy: 0.0844 - jaccard_coef_int: 0.6718 - binary_accuracy: 0.9631 - val_loss: 1.5217 - val_binary_crossentropy: 0.0570 - val_jaccard_coef_int: 0.6389 - val_binary_accuracy: 0.9904
Epoch 2/50
100/100 [==============================] - 9s 93ms/step - loss: 0.6358 - binary_crossentropy: 0.0238 - jaccard_coef_int: 0.7476 - binary_accuracy: 0.9951 - val_loss: 0.6500 - val_binary_crossentropy: 0.0563 - val_jaccard_coef_int: 0.6603 - val_binary_accuracy: 0.9911
Epoch 3/50
100/100 [==============================] - 9s 93ms/step - loss: 0.5290 - binary_crossentropy: 0.0239 - jaccard_coef_int: 0.7437 - binary_accuracy: 0.9953 - val_loss: 0.6110 - val_binary_crossentropy: 0.0604 - val_jaccard_coef_int: 0.6611 - val_binary_accuracy: 0.9911
Epoch 4/50
100/100 [==============================] - 9s 94ms/step - loss: 0.4824 - binary_crossentropy: 0.0287 - jaccard_coef_int: 0.7377 - binary_accuracy: 0.9

100/100 [==============================] - 9s 95ms/step - loss: 0.3710 - binary_crossentropy: 0.0486 - jaccard_coef_int: 0.7601 - binary_accuracy: 0.9956 - val_loss: 0.5479 - val_binary_crossentropy: 0.1217 - val_jaccard_coef_int: 0.6672 - val_binary_accuracy: 0.9912
Epoch 31/50
100/100 [==============================] - 9s 95ms/step - loss: 0.3790 - binary_crossentropy: 0.0456 - jaccard_coef_int: 0.7603 - binary_accuracy: 0.9959 - val_loss: 0.7957 - val_binary_crossentropy: 0.1430 - val_jaccard_coef_int: 0.5722 - val_binary_accuracy: 0.9862
Epoch 32/50
100/100 [==============================] - 9s 93ms/step - loss: 0.4015 - binary_crossentropy: 0.0521 - jaccard_coef_int: 0.7570 - binary_accuracy: 0.9955 - val_loss: 0.5247 - val_binary_crossentropy: 0.1186 - val_jaccard_coef_int: 0.6795 - val_binary_accuracy: 0.9915
Epoch 33/50
100/100 [==============================] - 9s 93ms/step - loss: 0.3048 - binary_crossentropy: 0.0437 - jaccard_coef_int: 0.7918 - binary_accuracy: 0.9959 - val_

Epoch 7/50
100/100 [==============================] - 9s 94ms/step - loss: 4.4819 - binary_crossentropy: 1.2559 - jaccard_coef_int: 0.3104 - binary_accuracy: 0.7945 - val_loss: 5.2124 - val_binary_crossentropy: 1.4552 - val_jaccard_coef_int: 0.1460 - val_binary_accuracy: 0.7942
Epoch 8/50
100/100 [==============================] - 10s 95ms/step - loss: 4.4909 - binary_crossentropy: 1.3156 - jaccard_coef_int: 0.3094 - binary_accuracy: 0.7939 - val_loss: 4.5200 - val_binary_crossentropy: 1.2901 - val_jaccard_coef_int: 0.2785 - val_binary_accuracy: 0.7980
Epoch 9/50
100/100 [==============================] - 9s 93ms/step - loss: 4.4038 - binary_crossentropy: 1.3431 - jaccard_coef_int: 0.3143 - binary_accuracy: 0.7981 - val_loss: 5.3252 - val_binary_crossentropy: 1.6663 - val_jaccard_coef_int: 0.2793 - val_binary_accuracy: 0.7477
Epoch 10/50
100/100 [==============================] - 9s 94ms/step - loss: 4.3562 - binary_crossentropy: 1.3712 - jaccard_coef_int: 0.3170 - binary_accuracy: 0.8

...

Epoch 19/50
100/100 [==============================] - 12s 117ms/step - loss: 2.5821 - binary_crossentropy: 0.5528 - jaccard_coef_int: 0.2424 - binary_accuracy: 0.9283 - val_loss: 3.7818 - val_binary_crossentropy: 0.7061 - val_jaccard_coef_int: 0.0924 - val_binary_accuracy: 0.9122
Epoch 20/50
100/100 [==============================] - 12s 117ms/step - loss: 2.5344 - binary_crossentropy: 0.5466 - jaccard_coef_int: 0.2466 - binary_accuracy: 0.9303 - val_loss: 3.5336 - val_binary_crossentropy: 0.8915 - val_jaccard_coef_int: 0.1751 - val_binary_accuracy: 0.8884
Epoch 21/50
100/100 [==============================] - 12s 118ms/step - loss: 2.4212 - binary_crossentropy: 0.5434 - jaccard_coef_int: 0.2665 - binary_accuracy: 0.9322 - val_loss: 3.4823 - val_binary_crossentropy: 0.7430 - val_jaccard_coef_int: 0.1368 - val_binary_accuracy: 0.9069
Epoch 22/50
100/100 [==============================] - 12s 117ms/step - loss: 2.3791 - binary_crossentropy: 0.5374 - jaccard_coef_int: 0.2713 - binary_acc

100/100 [==============================] - 12s 118ms/step - loss: 1.4874 - binary_crossentropy: 0.4690 - jaccard_coef_int: 0.4639 - binary_accuracy: 0.9553 - val_loss: 3.4901 - val_binary_crossentropy: 1.0183 - val_jaccard_coef_int: 0.1431 - val_binary_accuracy: 0.9034
Epoch 49/50
100/100 [==============================] - 12s 119ms/step - loss: 1.4790 - binary_crossentropy: 0.4731 - jaccard_coef_int: 0.4682 - binary_accuracy: 0.9553 - val_loss: 3.4694 - val_binary_crossentropy: 1.0590 - val_jaccard_coef_int: 0.1433 - val_binary_accuracy: 0.9044
Epoch 50/50
100/100 [==============================] - 12s 119ms/step - loss: 1.4587 - binary_crossentropy: 0.4716 - jaccard_coef_int: 0.4749 - binary_accuracy: 0.9557 - val_loss: 3.6232 - val_binary_crossentropy: 1.0807 - val_jaccard_coef_int: 0.1254 - val_binary_accuracy: 0.9034
0.9007252528901734
             precision    recall  f1-score   support

        0.0       0.93      0.97      0.95   1014360
        1.0       0.32      0.17      0.

Epoch 25/50
100/100 [==============================] - 12s 118ms/step - loss: 3.3095 - binary_crossentropy: 0.0629 - jaccard_coef_int: 0.1212 - binary_accuracy: 0.9892 - val_loss: 40.0500 - val_binary_crossentropy: 1.5102 - val_jaccard_coef_int: 5.9504e-16 - val_binary_accuracy: 0.6959
Epoch 26/50
100/100 [==============================] - 12s 118ms/step - loss: 2.9648 - binary_crossentropy: 0.0488 - jaccard_coef_int: 0.1284 - binary_accuracy: 0.9913 - val_loss: 28.1550 - val_binary_crossentropy: 0.0015 - val_jaccard_coef_int: 0.9125 - val_binary_accuracy: 0.9998
Epoch 27/50
100/100 [==============================] - 12s 117ms/step - loss: 3.5980 - binary_crossentropy: 0.0518 - jaccard_coef_int: 0.1360 - binary_accuracy: 0.9911 - val_loss: 31.3901 - val_binary_crossentropy: 0.0372 - val_jaccard_coef_int: 0.0625 - val_binary_accuracy: 0.9903
Epoch 28/50
100/100 [==============================] - 12s 117ms/step - loss: 3.2537 - binary_crossentropy: 0.0833 - jaccard_coef_int: 0.1313 - bin

100/100 [==============================] - 18s 185ms/step - loss: 5.1648 - binary_crossentropy: 0.0890 - jaccard_coef_int: 0.0477 - binary_accuracy: 0.9788 - val_loss: 7.0283 - val_binary_crossentropy: 0.0168 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9974
Epoch 2/50
100/100 [==============================] - 12s 120ms/step - loss: 3.7034 - binary_crossentropy: 0.0494 - jaccard_coef_int: 0.0744 - binary_accuracy: 0.9885 - val_loss: 7.0062 - val_binary_crossentropy: 0.0205 - val_jaccard_coef_int: 0.0500 - val_binary_accuracy: 0.9971
Epoch 3/50
100/100 [==============================] - 12s 120ms/step - loss: 2.9423 - binary_crossentropy: 0.0559 - jaccard_coef_int: 0.0947 - binary_accuracy: 0.9894 - val_loss: 7.5508 - val_binary_crossentropy: 0.0189 - val_jaccard_coef_int: 1.2311e-13 - val_binary_accuracy: 0.9971
Epoch 4/50
100/100 [==============================] - 12s 121ms/step - loss: 2.7711 - binary_crossentropy: 0.0472 - jaccard_coef_int: 0.1010 - binary_accuracy: 0.99

100/100 [==============================] - 12s 120ms/step - loss: 2.2709 - binary_crossentropy: 0.0651 - jaccard_coef_int: 0.1344 - binary_accuracy: 0.9929 - val_loss: 4.5123 - val_binary_crossentropy: 0.0576 - val_jaccard_coef_int: 0.0124 - val_binary_accuracy: 0.9933
Epoch 31/50
100/100 [==============================] - 12s 119ms/step - loss: 2.1311 - binary_crossentropy: 0.0670 - jaccard_coef_int: 0.1478 - binary_accuracy: 0.9933 - val_loss: 4.7688 - val_binary_crossentropy: 0.0310 - val_jaccard_coef_int: 0.0713 - val_binary_accuracy: 0.9967
Epoch 32/50
100/100 [==============================] - 12s 120ms/step - loss: 2.0275 - binary_crossentropy: 0.0638 - jaccard_coef_int: 0.1619 - binary_accuracy: 0.9939 - val_loss: 4.8948 - val_binary_crossentropy: 0.0350 - val_jaccard_coef_int: 0.0582 - val_binary_accuracy: 0.9965
Epoch 33/50
100/100 [==============================] - 12s 121ms/step - loss: 2.0896 - binary_crossentropy: 0.0630 - jaccard_coef_int: 0.1545 - binary_accuracy: 0.994

In [16]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/relutest/80_80_16/results_prelim_unet_pl_c24pl'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/planetlabs/ClusterRaster24_PlanetLabs.tif'],dict_label)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
#     for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
#         if x[1] < 400*36:
#             list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        del ts_gen
        del X_train, X_val, X_test, Y_train, Y_val, Y_test
        del Y_predict
        K.clear_session()          
        print('\n')

unique labels found:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
labels that do not meet threshold
[]
final labels
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


label: 0
{0: 1.0}
composition with label != 0
Y_train 0.9307568438003221 0.0945511272141707
composition with label != 0
Y_val 0.9571428571428572 0.08802008928571428
composition with label != 0
Y_test 0.9248554913294798 0.08844924132947977
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., callbacks=[<keras.ca..., validation_data=(array([[[..., steps_per_epoch=100, verbose=1, epochs=50)`


Epoch 1/50
100/100 [==============================] - 16s 165ms/step - loss: 3.3394 - binary_crossentropy: 0.2243 - jaccard_coef_int: 0.4364 - binary_accuracy: 0.9078 - val_loss: 3.8405 - val_binary_crossentropy: 0.3339 - val_jaccard_coef_int: 0.1079 - val_binary_accuracy: 0.9144
Epoch 2/50
100/100 [==============================] - 10s 99ms/step - loss: 2.0467 - binary_crossentropy: 0.1905 - jaccard_coef_int: 0.5206 - binary_accuracy: 0.9353 - val_loss: 2.1947 - val_binary_crossentropy: 0.2568 - val_jaccard_coef_int: 0.4004 - val_binary_accuracy: 0.9299
Epoch 3/50
100/100 [==============================] - 10s 101ms/step - loss: 1.8905 - binary_crossentropy: 0.2137 - jaccard_coef_int: 0.5238 - binary_accuracy: 0.9363 - val_loss: 1.8801 - val_binary_crossentropy: 0.2619 - val_jaccard_coef_int: 0.4794 - val_binary_accuracy: 0.9353
Epoch 4/50
100/100 [==============================] - 10s 97ms/step - loss: 1.7028 - binary_crossentropy: 0.2304 - jaccard_coef_int: 0.5398 - binary_accuracy:

100/100 [==============================] - 10s 100ms/step - loss: 0.6726 - binary_crossentropy: 0.1570 - jaccard_coef_int: 0.7711 - binary_accuracy: 0.9749 - val_loss: 0.5960 - val_binary_crossentropy: 0.1279 - val_jaccard_coef_int: 0.7790 - val_binary_accuracy: 0.9790
Epoch 31/50
100/100 [==============================] - 10s 98ms/step - loss: 0.6741 - binary_crossentropy: 0.1588 - jaccard_coef_int: 0.7675 - binary_accuracy: 0.9751 - val_loss: 0.5922 - val_binary_crossentropy: 0.1326 - val_jaccard_coef_int: 0.7793 - val_binary_accuracy: 0.9790
Epoch 32/50
100/100 [==============================] - 11s 105ms/step - loss: 0.6438 - binary_crossentropy: 0.1509 - jaccard_coef_int: 0.7758 - binary_accuracy: 0.9763 - val_loss: 0.5635 - val_binary_crossentropy: 0.1231 - val_jaccard_coef_int: 0.7911 - val_binary_accuracy: 0.9799
Epoch 33/50
100/100 [==============================] - 10s 99ms/step - loss: 0.6483 - binary_crossentropy: 0.1551 - jaccard_coef_int: 0.7746 - binary_accuracy: 0.9761 

Epoch 7/50
100/100 [==============================] - 12s 118ms/step - loss: 1.8790 - binary_crossentropy: 0.2536 - jaccard_coef_int: 0.3611 - binary_accuracy: 0.9491 - val_loss: 2.2378 - val_binary_crossentropy: 0.2823 - val_jaccard_coef_int: 0.2826 - val_binary_accuracy: 0.9414
Epoch 8/50
100/100 [==============================] - 12s 116ms/step - loss: 1.7703 - binary_crossentropy: 0.2524 - jaccard_coef_int: 0.3836 - binary_accuracy: 0.9518 - val_loss: 1.6335 - val_binary_crossentropy: 0.2455 - val_jaccard_coef_int: 0.4504 - val_binary_accuracy: 0.9508
Epoch 9/50
100/100 [==============================] - 12s 117ms/step - loss: 1.6705 - binary_crossentropy: 0.2464 - jaccard_coef_int: 0.4058 - binary_accuracy: 0.9543 - val_loss: 1.6552 - val_binary_crossentropy: 0.2700 - val_jaccard_coef_int: 0.4476 - val_binary_accuracy: 0.9495
Epoch 10/50
100/100 [==============================] - 12s 117ms/step - loss: 1.6337 - binary_crossentropy: 0.2477 - jaccard_coef_int: 0.4182 - binary_accura

...

100/100 [==============================] - 12s 120ms/step - loss: 0.7427 - binary_crossentropy: 0.1745 - jaccard_coef_int: 0.7319 - binary_accuracy: 0.9738 - val_loss: 0.7100 - val_binary_crossentropy: 0.1502 - val_jaccard_coef_int: 0.7269 - val_binary_accuracy: 0.9762
Epoch 40/50
100/100 [==============================] - 12s 118ms/step - loss: 0.7234 - binary_crossentropy: 0.1702 - jaccard_coef_int: 0.7369 - binary_accuracy: 0.9745 - val_loss: 0.7347 - val_binary_crossentropy: 0.1608 - val_jaccard_coef_int: 0.7193 - val_binary_accuracy: 0.9752
Epoch 41/50
100/100 [==============================] - 12s 119ms/step - loss: 0.7016 - binary_crossentropy: 0.1644 - jaccard_coef_int: 0.7438 - binary_accuracy: 0.9753 - val_loss: 0.6467 - val_binary_crossentropy: 0.1328 - val_jaccard_coef_int: 0.7453 - val_binary_accuracy: 0.9785
Epoch 42/50
100/100 [==============================] - 12s 121ms/step - loss: 0.7103 - binary_crossentropy: 0.1719 - jaccard_coef_int: 0.7405 - binary_accuracy: 0.975

Epoch 16/50
100/100 [==============================] - 12s 116ms/step - loss: 0.9330 - binary_crossentropy: 0.1373 - jaccard_coef_int: 0.5819 - binary_accuracy: 0.9766 - val_loss: 1.1614 - val_binary_crossentropy: 0.1191 - val_jaccard_coef_int: 0.4383 - val_binary_accuracy: 0.9801
Epoch 17/50
100/100 [==============================] - 11s 114ms/step - loss: 0.9329 - binary_crossentropy: 0.1424 - jaccard_coef_int: 0.5823 - binary_accuracy: 0.9765 - val_loss: 1.0411 - val_binary_crossentropy: 0.1053 - val_jaccard_coef_int: 0.4756 - val_binary_accuracy: 0.9824
Epoch 18/50
100/100 [==============================] - 11s 114ms/step - loss: 0.9180 - binary_crossentropy: 0.1416 - jaccard_coef_int: 0.5877 - binary_accuracy: 0.9768 - val_loss: 1.1156 - val_binary_crossentropy: 0.1168 - val_jaccard_coef_int: 0.4472 - val_binary_accuracy: 0.9813
Epoch 19/50
100/100 [==============================] - 11s 114ms/step - loss: 0.8901 - binary_crossentropy: 0.1359 - jaccard_coef_int: 0.5940 - binary_acc

100/100 [==============================] - 12s 117ms/step - loss: 0.5542 - binary_crossentropy: 0.0937 - jaccard_coef_int: 0.7247 - binary_accuracy: 0.9861 - val_loss: 0.8428 - val_binary_crossentropy: 0.1228 - val_jaccard_coef_int: 0.5415 - val_binary_accuracy: 0.9856
Epoch 46/50
100/100 [==============================] - 11s 113ms/step - loss: 0.5329 - binary_crossentropy: 0.0904 - jaccard_coef_int: 0.7368 - binary_accuracy: 0.9864 - val_loss: 0.5364 - val_binary_crossentropy: 0.0727 - val_jaccard_coef_int: 0.7020 - val_binary_accuracy: 0.9892
Epoch 47/50
100/100 [==============================] - 12s 118ms/step - loss: 0.5209 - binary_crossentropy: 0.0890 - jaccard_coef_int: 0.7425 - binary_accuracy: 0.9866 - val_loss: 0.5168 - val_binary_crossentropy: 0.0693 - val_jaccard_coef_int: 0.7106 - val_binary_accuracy: 0.9897
Epoch 48/50
100/100 [==============================] - 12s 117ms/step - loss: 0.5558 - binary_crossentropy: 0.1000 - jaccard_coef_int: 0.7290 - binary_accuracy: 0.985

Epoch 22/50
100/100 [==============================] - 12s 118ms/step - loss: 1.0616 - binary_crossentropy: 0.2662 - jaccard_coef_int: 0.6942 - binary_accuracy: 0.9575 - val_loss: 1.6081 - val_binary_crossentropy: 0.4697 - val_jaccard_coef_int: 0.5829 - val_binary_accuracy: 0.9338
Epoch 23/50
100/100 [==============================] - 12s 118ms/step - loss: 1.0416 - binary_crossentropy: 0.2644 - jaccard_coef_int: 0.6962 - binary_accuracy: 0.9586 - val_loss: 0.9934 - val_binary_crossentropy: 0.2483 - val_jaccard_coef_int: 0.7265 - val_binary_accuracy: 0.9589
Epoch 24/50
100/100 [==============================] - 12s 118ms/step - loss: 0.9905 - binary_crossentropy: 0.2513 - jaccard_coef_int: 0.7087 - binary_accuracy: 0.9606 - val_loss: 0.9153 - val_binary_crossentropy: 0.2259 - val_jaccard_coef_int: 0.7403 - val_binary_accuracy: 0.9623
Epoch 25/50
100/100 [==============================] - 12s 118ms/step - loss: 1.0021 - binary_crossentropy: 0.2612 - jaccard_coef_int: 0.7026 - binary_acc

             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99    980995
        1.0       0.89      0.91      0.90    126205

avg / total       0.98      0.98      0.98   1107200



